# Finding Shortest path using Q-learning
## Importing libraries

In [ ]:
import networkx as nx
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

## Transforming the grid into a graph

One way to model this problem, is to consider it as a graph  $G(N,A)$, that each cell of grid represents a vertex in $N$ whose connections with neighbor cells can be considered as the edges in $A$.  For example cell $5$ is neighbor with cells $4,6$. Note that, although a cell might be a neighbour for other cells, but those neighbors might not be reachable from the cell. I still consider arcs between the cell and unreachable neighbours, however, the reward for choosing those arcs are highly negative; I call these arcs as "broken arcs".

In the following example, the agent starts at cell $0$ and wants to reach, the target $14$. The set of broken arcs is as follows:
$\{(2, 6),
 (1, 5),
 (7, 11),
 (10, 14),
 (5, 9),
 (6, 2),
 (5, 1),
 (11, 7),
 (14, 10),
 (9, 5)  \}$


In [ ]:
n_r,n_c = 4,4 #num_row, num_col
target = 15
start = 0
G_ = nx.grid_2d_graph(n_r,n_c)
broken_arcs_ = [ ((0,2), (1,2)),((0,1), (1,1)),((1,3), (2,3)),((2,2), (3,2)),((1,1),(2,1))]
broken_arcs_ +=[(j,i) for i,j in broken_arcs_]
broken_arcs = [(list(G_.nodes).index(i),list(G_.nodes).index(j)) for i,j in broken_arcs_]
G = nx.Graph()
G.add_nodes_from([i for i in range(len(G_.nodes))])
G.add_edges_from([(list(G_.nodes).index(i),list(G_.nodes).index(j)) for i,j in G_.edges])

plt.figure(figsize=(6,6))
pos = {list(G_.nodes()).index((x,y)):(y,-x) for x,y in G_.nodes()}
nx.draw(G.edge_subgraph([(i,j) for (i,j) in G.edges if (i,j) not in broken_arcs]), pos=pos,
        node_color=['green' if i not in [start,target] else ('red' if i==target else 'blue') for i in G.nodes()],
        with_labels=True,
        node_size=600,
        node_shape='s')

## Reward definition

Reward for choosing non-broken arcs leading to target node, from its neighbor is $100$. The reward of choosing a broken arc is $-100$. Choosing the rest of acs is rewarded $0$. Similarly, we set the q-value for broken arcs to $-100$.

In [ ]:

R = np.zeros((len(G.nodes),len(G.nodes)))
Q = -100*np.ones_like(R)
for i,j in G.edges:
    R[i,j]= -100  if (i,j) in broken_arcs else 0
    R[j,i]= -100  if (i,j) in broken_arcs else 0
    if j==target:
        R[i,j] = 100
    if (i,j) not in broken_arcs:
        Q[i,j] = 0
        Q[j,i] = 0


Based on a given state, i.e. **current location of the agent**, the following function will determine the agent's action. This function, besides the the state, also requires **epsilon $\epsilon$**, determining the possibility of exploration vs. exploitation. In other words, the agent will explore by the chance of $\epsilon$ and exploit (choosing best greedy action) by probability of  $1-\epsilon$.

In [ ]:
def next_action(state,eps):
    if np.random.rand()<=eps:
        return int(np.random.choice(list(G[state])))
    else:
        return int(np.random.choice(np.where(Q[state]==Q[state].max())[0]))

In every steps $t$, we require to update the $Q(S_t,A_t)$.
${\displaystyle Q^{new}(S_{t},A_{t})\leftarrow (1-\underbrace {\alpha } _{\text{learning rate}})\cdot \underbrace {Q(S_{t},A_{t})} _{\text{current value}}+\underbrace {\alpha } _{\text{learning rate}}\cdot {\bigg (}\underbrace {\underbrace {r(S_t,A_t)} _{\text{reward}}+\underbrace {\gamma } _{\text{discount factor}}\cdot \underbrace {\max _{a}Q(S_{t+1},a)} _{\text{estimate of optimal future value}}} _{\text{new value (temporal difference target)}}{\bigg )}}$

Note that ${\displaystyle Q^{new}(S_{t},A_{t})}$ is the sum of three factors:
1. $ {\displaystyle (1-\alpha )Q(S_{t},A_{t})}$: the current value (weighted by one minus the learning rate)
2. ${\displaystyle \alpha \,R_{t+1}}$: the reward ${\displaystyle R_{t+1}}$ to obtain if action ${\displaystyle A_{t}}$ is taken when in state ${\displaystyle S_{t}} $ (weighted by learning rate)
3. ${\displaystyle \alpha \gamma \max _{a}Q(S_{t+1},a)}$: the maximum reward that can be obtained from state ${\displaystyle S_{t+1}}$ (weighted by learning rate and discount factor)

An episode of the algorithm ends when state ${\displaystyle S_{t+1}}$ is a final or terminal state.



In [ ]:
def Update_Q(state,action,alpha,gamma):
    next_action = np.random.choice(np.where(Q[action,:]==Q[action,:].max())[0])
    Q[state,action] = (1-alpha)*Q[state,action]+alpha*(R[state,action]+gamma*Q[action,next_action])



Now, we bundle every functions together and allows the agent to perform it $1000$ times. Later, we modify the $\epsilon$, $\gamma$ and $\alpha$ to examine the quality of q-values approximation.

In [ ]:
def get_path(start,target):
    q_val = 0
    P = [start]
    current_node = start
    while current_node!=target:
        next_node = np.argmax(Q[current_node,:])
        P.append(next_node)

        q_val+=Q[current_node,next_node]
        current_node = next_node
    return P
def QLearn(eps,alpha,gamma):
    Q_best = [0]
    for num_epochs in tqdm(range(1000)):
        state = 0
        while True:
            action = next_action(state,eps)
            Update_Q(state,action,alpha,gamma)
            Q_best.append(Q[start,:].max())
            state = action
            if state == target:
                break
        # Q_best.append(get_path(start,target)[1])
    return Q_best

Q_best = QLearn(0.2,.8,.5)

In [ ]:
plt.style.use('ggplot')
plt.plot(Q_best)
plt.xlabel("#Iterations");
plt.ylabel("V(start)");


The Q-value I am plotting here is the maximum of $V(start)$. Techniqually, we want to estimate $V(start)$, via the Q-learning algorithm

In [ ]:
get_path(start,target)

## Testing different parameter

The default values for $\epsilon = 0.2,\ \alpha=0.8,\ \gamma=0.8 $

Due to simplicity of network, the change of hyperperparameter is not making any differences.
### Optimizing $\epsilon\in \{0.1,0.2,\ldots, 0.7\}$



In [ ]:
plt.style.use('ggplot')
for eps in range(1,8):
    R = np.zeros((len(G.nodes),len(G.nodes)))
    Q = -100*np.ones_like(R)
    for i,j in G.edges:
        R[i,j]= -100  if (i,j) in broken_arcs else 0
        R[j,i]= -100  if (i,j) in broken_arcs else 0
        if j==target:
            R[i,j] = 100
        if (i,j) not in broken_arcs:
            Q[i,j] = 0
            Q[j,i] = 0
    plt.plot(QLearn(eps*0.1,.8,.8),label=r'$\epsilon= '+ str(np.round(eps*0.1,1))+' $')
plt.xlabel("#Iterations");
plt.ylabel("Q");
plt.legend()

### Optimizing $\alpha\in \{0.1,\ 0.2,\ \ldots,\ 0.9\}$

In [ ]:
plt.style.use('ggplot')
for a in range(1,10):
    R = np.zeros((len(G.nodes),len(G.nodes)))
    Q = -100*np.ones_like(R)
    for i,j in G.edges:
        R[i,j]= -100  if (i,j) in broken_arcs else 0
        R[j,i]= -100  if (i,j) in broken_arcs else 0
        if j==target:
            R[i,j] = 100
        if (i,j) not in broken_arcs:
            Q[i,j] = 0
            Q[j,i] = 0
    plt.plot(QLearn(0.2,a*0.1,.8),label=r'$\alpha= '+ str(np.round(a*0.1,1))+' $')
plt.xlabel("#Iterations");
plt.ylabel("Q");
plt.legend()

### Optimizing $\gamma \in \{0, 0.1,\ 0.2,\ \ldots,\ 0.9\}$

In [ ]:
plt.style.use('ggplot')
for g in range(5,10):
    R = np.zeros((len(G.nodes),len(G.nodes)))
    Q = -100*np.ones_like(R)
    for i,j in G.edges:
        R[i,j]= -100  if (i,j) in broken_arcs else 0
        R[j,i]= -100  if (i,j) in broken_arcs else 0
        if j==target:
            R[i,j] = 100
        if (i,j) not in broken_arcs:
            Q[i,j] = 0
            Q[j,i] = 0
    plt.plot(QLearn(0.2,0.8,g*0.1),label=r'$\gamma= '+ str(np.round(g*0.1,1))+' $')
plt.xlabel("#Iterations");
plt.ylabel("Q");
plt.legend()


1. **Gamma (γ)**:
   - Gamma is the discount factor that represents the importance of future rewards in the Q-learning algorithm. It determines how much importance the agent places on future rewards when making decisions.
   - A higher gamma value means that the agent values long-term rewards more and is willing to sacrifice short-term gains for higher cumulative rewards over time.
   - Conversely, a lower gamma value means that the agent focuses more on immediate rewards and may exhibit more myopic behavior, prioritizing short-term gains over long-term strategies.

2. **Alpha (α)**:
   - Alpha is the learning rate that controls the extent to which the Q-values are updated based on new information obtained during each iteration of the Q-learning algorithm.
   - A higher alpha value means that the agent gives more weight to recent experiences and adjusts its Q-values more aggressively, leading to faster learning but potentially less stability.
   - On the other hand, a lower alpha value results in slower learning but may lead to more stable and consistent convergence of Q-values over time.

3. **Epsilon (ε)**:
   - Epsilon is the exploration-exploitation parameter that determines the balance between exploration (trying new actions to discover potentially better strategies) and exploitation (choosing actions based on current knowledge to maximize immediate rewards).
   - A higher epsilon value encourages more exploration, as the agent is more likely to choose random actions rather than relying solely on its current knowledge (Q-values).
   - Conversely, a lower epsilon value promotes more exploitation, as the agent tends to choose actions based on its learned Q-values, potentially leading to more optimal but less exploratory behavior.



## Reference:


1.   [Finding Shortest Path using Q-Learning Algorithm](https://towardsdatascience.com/finding-shortest-path-using-q-learning-algorithm-1c1f39e89505)
2.   [Q-learning: A Complete Example in Python](https://www.youtube.com/watch?v=iKdlKYG78j4)
3. [Q-learning](https://en.wikipedia.org/wiki/Q-learning)

